# Data treatment
Nessa seção, iremos filtrar os dados do arquivo CSV que geramos anteriormente em apenas os CIDs correspondentes a suicídio.


In [14]:
# Imports
import csv
import json
import pandas as pd

In [2]:
# Dataset com os dados de suicídio de 2011 a 2015
df_suicidio = pd.read_csv('data/suicidios.csv', sep=';')

In [3]:
df_suicidio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52537 entries, 0 to 52536
Data columns (total 24 columns):
DTOBITO       52537 non-null int64
DTNASC        52251 non-null float64
SEXO          52537 non-null int64
RACACOR       50640 non-null float64
ESTCIV        50658 non-null float64
OCUP          39613 non-null float64
CODMUNRES     52537 non-null int64
LOCOCOR       52537 non-null int64
CODMUNOCOR    52537 non-null int64
IDADEMAE      0 non-null float64
ESCMAE        0 non-null float64
OCUPMAE       0 non-null float64
QTDFILVIVO    0 non-null float64
QTDFILMORT    0 non-null float64
GRAVIDEZ      0 non-null float64
GESTACAO      0 non-null float64
CAUSABAS      52537 non-null object
LINHAA        50391 non-null object
LINHAB        35987 non-null object
LINHAC        9297 non-null object
LINHAD        3505 non-null object
LINHAII       3842 non-null object
CIRCOBITO     52497 non-null float64
FONTE         44806 non-null float64
dtypes: float64(13), int64(5), object(6)
memory us

In [4]:
# Visto que temos algumas colunas não preenchidas, iremos excluir elas
df_suicidio = df_suicidio.drop(columns=['IDADEMAE', 'ESCMAE', 'OCUPMAE',
                                        'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'GESTACAO'])

Códigos do IBGE para cada Unidade Federal (UF):

| Região       | Código | Unidade da Federação | UF |
| ------------ | ------ | -------------------- | -- |
| Norte        | 11     | Rondônia             | RO |
|              | 12     | Acre                 | AC |
|              | 13     | Amazonas             | AM |
|              | 14     | Roraima              | RR |
|              | 15     | Pará                 | PA |
|              | 16     | Amapá                | AP |
|              | 17     | Tocantins            | TO |
| Nordeste     | 21     | Maranhão             | MA |
|              | 22     | Piauí                | PI |
|              | 23     | Ceará                | CE |
|              | 24     | Rio Grande do Norte  | RN |
|              | 25     | Paraíba              | PB |
|              | 26     | Pernambuco           | PE |
|              | 27     | Alagoas              | AL |
|              | 28     | Sergipe              | SE |
|              | 29     | Bahia                | BA |
| Sudeste      | 31     | Minas Gerais         | MG |
|              | 32     | Espírito Santo       | ES |
|              | 33     | Rio de Janeiro       | RJ |
|              | 35     | São Paulo            | SP |
| Sul          | 41     | Paraná               | PR |
|              | 42     | Santa Catarina       | SC |
|              | 43     | Rio Grande do Sul    | RS |
| Centro-Oeste | 50     | Mato Grosso do Sul   | MS |
|              | 51     | Mato Grosso          | MT |
|              | 52     | Goiás                | GO |
|              | 53     | Distrito Federal     | DF |


In [5]:
# Criamos uma coluna com o código da UF
df_suicidio['UF'] = df_suicidio['CODMUNOCOR'].apply(lambda x: str(x)[0:2])

# Criamos outra coluna com o ano da ocorrência
df_suicidio['ANOOCOR'] = df_suicidio['DTOBITO'].apply(lambda x: str(x)[-4:])

In [25]:
### Ocorrências por ano e UF
df_ano_uf = df_suicidio.groupby(['ANOOCOR', 'UF'])['DTOBITO'].count()

# Gera arquivo JSON
with open('data/ano_uf.json', 'w') as json_file:
    json_file.write( df_ano_uf.reset_index().to_json(orient='records') )
    json_file.close()

# Organiza direito o JSON
json_organizado = dict()

with open('data/ano_uf.json') as file:
    json_data = json.load(file)
    for dt in json_data:
        if dt['ANOOCOR'] in json_organizado:
            json_organizado[ dt['ANOOCOR'] ].update({ dt['UF'] : dt['DTOBITO'] })
        else:
            json_organizado.update({ dt['ANOOCOR'] : dict() })

# Gera o arquivo JSON adequado
with open('data/ano_uf.json', 'w') as file:
    json.dump(json_organizado, file)

In [7]:
### Pegar a quantidade de ocorrências por data
datas_obito = dict()

# Pego a quantidade de registros de cada data
for dtobito in df_suicidio.DTOBITO:
    if dtobito in datas_obito:
        datas_obito[ dtobito ] += 1
    else:
        datas_obito[ dtobito ] = 1
        
# Modifico as chaves do dicionario
for dtobito in df_suicidio.DTOBITO.unique():
    dtobito = str(dtobito)
    if len(dtobito) == 7:
        new_dtobito = '-'.join([dtobito[3:], dtobito[1:3], ('0'+dtobito[0])])
    else:
        new_dtobito = '-'.join([dtobito[4:], dtobito[2:4], dtobito[0:2]])
        
    datas_obito[new_dtobito] = datas_obito.pop(int(dtobito))

# Exportar para CSV
with open('data/ocor_data.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['date', 'ocor'])
    for key, value in datas_obito.items():
        writer.writerow([str(key), str(value)])

Para análisar os suicídios em feriados, escolhemos 8 feriados:

| Data (m-d) | Feriado |
| ----------:|:------- |
|    01-01   | Confraternização Universal |
|    04-21   | Tiradentes |
|    05-01   | Dia do Trabalhador |
|    09-07   | Dia da Pátria |
|    10-12   | Nossa Senhora Aparecida |
|    11-02   | Finados |
|    11-15   | Proclamação da República |
|    12-25   | Natal |

In [51]:
### Suicídios nos feriados
feriados = ['01-01', '04-21', '05-01', '09-07', '10-12', '11-02', '11-15', '12-25']
feriados